In [313]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy

In [314]:
def generate_random_graph(num_nodes, probability):
    graph = [[0] * num_nodes for _ in range(num_nodes)]
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if random.random() < probability:
                graph[i][j] = 1
                graph[j][i] = 1

    return graph

def generate_random_solution(num_nodes):
    nodes = list(range(num_nodes))
    random.shuffle(nodes)
    return nodes

def calculate_total_edge_length(graph, solution):
    total_length = 0
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            if graph[i][j] == 1:
                position_i = solution.index(i)
                position_j = solution.index(j)
                total_length += abs(position_i - position_j)
    return total_length

# Example usage:
num_nodes = 10
edge_probability = 0.5

# Generate a random graph
graph = generate_random_graph(num_nodes, edge_probability)

# Generate a random solution
random_solution = generate_random_solution(num_nodes)

# Calculate the total edge length for the random solution
total_edge_length = calculate_total_edge_length(graph, random_solution)

print("Random Graph (Adjacency Matrix):")
for row in graph:
    print(row)

print("\nRandom Solution (Permutation):", random_solution)
print("Total Edge Length:", total_edge_length)

Random Graph (Adjacency Matrix):
[0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 1, 1, 1, 0]
[0, 1, 0, 0, 0, 0, 1, 0, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 1]
[0, 1, 1, 1, 0, 0, 0, 1, 1, 0]
[0, 1, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 0]

Random Solution (Permutation): [0, 3, 5, 6, 7, 9, 2, 8, 4, 1]
Total Edge Length: 73


In [315]:
import itertools

def brute_force(graph):
    num_nodes = len(graph)
    best_solution = None
    best_length = float('inf')

    # Generate all permutations of nodes
    all_permutations = itertools.permutations(range(num_nodes))

    # Check total edge length for each permutation
    for solution in all_permutations:
        total_length = calculate_total_edge_length(graph, solution)
        if total_length < best_length:
            best_length = total_length
            best_solution = solution

    return best_solution, best_length

In [316]:
brute_force(graph)

((4, 3, 7, 6, 1, 0, 8, 2, 5, 9), 43)

In [317]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [318]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [319]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [320]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [321]:
def local_search_first_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [322]:
def local_search_best_improvement(graph, random_solution, value, num_iters, change_func):
    pass

In [323]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 1
[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 2
[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 3
[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 4
[0, 3, 5, 8, 7, 9, 2, 6, 4, 1] 71 5
[0, 3, 5, 8, 7, 9, 2, 6, 4, 1] 71 6
[0, 3, 5, 8, 7, 9, 2, 6, 4, 1] 71 7
[0, 3, 5, 8, 7, 9, 2, 6, 4, 1] 71 8
[0, 3, 5, 8, 7, 9, 6, 2, 4, 1] 70 9
[0, 3, 5, 8, 4, 9, 6, 2, 7, 1] 62 10
[0, 3, 5, 8, 4, 9, 6, 2, 7, 1] 62 11
[0, 4, 5, 8, 3, 9, 6, 2, 7, 1] 54 12
[0, 4, 5, 8, 3, 9, 6, 2, 7, 1] 54 13
[0, 4, 5, 8, 3, 9, 6, 2, 7, 1] 54 14
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 15
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 16
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 17
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 18
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 19
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 20
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 21
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 22
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 23
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 24
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 25
[0, 4, 5, 9, 3, 8, 6, 2, 7, 1] 52 26
[0, 4, 5, 9

([9, 4, 5, 0, 3, 8, 2, 6, 1, 7], 47, 110)

In [324]:
local_search(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 1
[0, 3, 5, 6, 7, 9, 4, 8, 2, 1] 71 2
[0, 3, 5, 6, 8, 4, 9, 7, 2, 1] 69 3
[0, 3, 5, 6, 8, 4, 7, 9, 2, 1] 68 4
[0, 3, 5, 6, 8, 4, 7, 9, 2, 1] 68 5
[0, 3, 5, 9, 7, 4, 8, 6, 2, 1] 64 6
[0, 1, 2, 6, 8, 4, 7, 9, 5, 3] 60 7
[0, 1, 2, 6, 8, 4, 3, 5, 9, 7] 59 8
[0, 1, 2, 6, 8, 4, 3, 5, 9, 7] 59 9
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 10
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 11
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 12
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 13
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 14
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 15
[0, 9, 5, 3, 4, 8, 6, 2, 1, 7] 54 16
[9, 0, 5, 3, 4, 8, 6, 2, 1, 7] 52 17
[9, 0, 5, 3, 4, 8, 6, 2, 1, 7] 52 18
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 19
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 20
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 21
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 22
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 23
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 24
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 25
[9, 0, 5, 3, 4, 8, 2, 6, 1, 7] 51 26
[9, 0, 5, 3

([9, 4, 3, 5, 0, 8, 2, 6, 1, 7], 47, 30)

In [325]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i



In [326]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_inverse)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
[0, 3, 5, 6, 2, 9, 8, 1, 4, 7] 71 1
[9, 2, 6, 5, 3, 0, 8, 1, 4, 7] 58 2
[9, 2, 6, 5, 3, 1, 8, 0, 4, 7] 56 3
[9, 2, 6, 5, 3, 8, 1, 0, 4, 7] 54 4
[9, 2, 6, 5, 3, 8, 1, 0, 4, 7] 54 5
[9, 2, 6, 5, 1, 8, 3, 0, 7, 4] 53 6
[9, 2, 6, 5, 1, 8, 7, 0, 3, 4] 51 7
[9, 2, 6, 5, 1, 8, 7, 0, 3, 4] 51 8
[9, 2, 6, 5, 1, 8, 7, 0, 3, 4] 51 9
[9, 2, 6, 5, 1, 8, 7, 0, 3, 4] 51 10
[9, 2, 6, 5, 1, 8, 7, 0, 3, 4] 51 11
[9, 2, 5, 6, 1, 8, 7, 0, 3, 4] 50 12
[9, 2, 5, 6, 1, 8, 7, 0, 3, 4] 50 13
[9, 2, 5, 6, 1, 8, 7, 0, 3, 4] 50 14
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 15
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 16
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 17
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 18
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 19
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 20
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 21
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 22
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 23
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 24
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 25
[9, 2, 5, 8, 1, 6, 7, 0, 3, 4] 46 26
[9, 2, 5, 8

([9, 2, 5, 8, 0, 1, 6, 7, 3, 4], 43, 77)

In [327]:
local_search_with_permutation(graph, random_solution, total_edge_length, 1000, change_func=make_change_swap)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
[0, 3, 5, 4, 7, 9, 2, 8, 6, 1] 62 1
[0, 3, 5, 4, 7, 9, 1, 8, 6, 2] 59 2
[0, 3, 5, 4, 8, 9, 2, 1, 6, 7] 56 3
[0, 3, 5, 4, 8, 1, 6, 2, 7, 9] 55 4
[0, 3, 5, 4, 8, 1, 6, 2, 7, 9] 55 5
[0, 3, 5, 4, 8, 1, 6, 2, 7, 9] 55 6
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 7
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 8
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 9
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 10
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 11
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 12
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 13
[0, 3, 5, 4, 8, 6, 1, 7, 2, 9] 54 14
[0, 5, 3, 4, 8, 6, 1, 7, 2, 9] 52 15
[0, 5, 3, 4, 8, 6, 1, 7, 2, 9] 52 16
[0, 5, 3, 4, 8, 6, 1, 7, 2, 9] 52 17
[0, 5, 3, 4, 8, 6, 1, 7, 2, 9] 52 18
[0, 5, 3, 4, 8, 6, 1, 7, 2, 9] 52 19
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 20
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 21
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 22
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 23
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 24
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 25
[4, 5, 3, 0, 8, 6, 1, 7, 2, 9] 48 26
[4, 3, 5, 0

([4, 3, 5, 0, 8, 6, 1, 7, 2, 9], 46, 26)

In [328]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        elif random.random() < 1 / (i**0.5):
            print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [329]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73
divs
[0, 3, 2, 6, 7, 9, 5, 8, 4, 1] 83
[7, 3, 2, 6, 0, 9, 5, 8, 4, 1] 71
[7, 3, 2, 6, 0, 9, 5, 8, 4, 1] 71
[7, 3, 2, 6, 0, 9, 5, 8, 4, 1] 71
[7, 3, 2, 6, 9, 0, 5, 8, 4, 1] 67
[7, 3, 2, 6, 9, 0, 5, 8, 4, 1] 67
[7, 3, 2, 6, 9, 0, 5, 8, 4, 1] 67
[7, 3, 2, 6, 9, 0, 5, 8, 1, 4] 64
[7, 3, 2, 6, 9, 1, 5, 8, 0, 4] 53
[7, 3, 2, 6, 9, 1, 5, 8, 0, 4] 53
[7, 3, 6, 2, 9, 1, 5, 8, 0, 4] 50
[7, 3, 6, 2, 9, 1, 5, 8, 0, 4] 50
divs
[7, 5, 6, 2, 9, 1, 3, 8, 0, 4] 64
[7, 5, 6, 2, 9, 1, 3, 8, 0, 4] 64
[7, 2, 6, 5, 9, 1, 3, 8, 0, 4] 62
divs
[1, 2, 6, 5, 9, 7, 3, 8, 0, 4] 64
[1, 2, 6, 5, 9, 7, 3, 8, 0, 4] 64
[1, 2, 6, 5, 9, 7, 3, 8, 0, 4] 64
[1, 2, 6, 5, 9, 7, 3, 8, 0, 4] 64
[1, 2, 6, 9, 5, 7, 3, 8, 0, 4] 60
[1, 2, 6, 9, 5, 7, 3, 8, 0, 4] 60
[1, 2, 6, 9, 5, 7, 3, 8, 0, 4] 60
[7, 2, 6, 9, 5, 1, 3, 8, 0, 4] 58
divs
[2, 7, 6, 9, 5, 1, 3, 8, 0, 4] 59
[2, 7, 6, 9, 5, 1, 3, 8, 0, 4] 59
[2, 7, 6, 9, 5, 1, 3, 8, 0, 4] 59
[2, 7, 6, 9, 5, 1, 3, 8, 0, 4] 59
[2, 9, 6, 7, 5, 1, 3, 8, 0, 

([7, 3, 4, 6, 1, 0, 8, 5, 2, 9], 45, 279)

In [330]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

[0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73
divs
[0, 3, 8, 2, 9, 7, 6, 5, 4, 1] 86
[0, 3, 8, 1, 4, 5, 6, 7, 9, 2] 68
divs
[0, 3, 8, 6, 5, 4, 1, 7, 9, 2] 68
[0, 5, 6, 8, 3, 4, 1, 7, 9, 2] 64
[0, 5, 6, 8, 3, 4, 1, 7, 9, 2] 64
divs
[0, 6, 5, 8, 3, 4, 1, 7, 9, 2] 67
divs
[0, 6, 5, 8, 9, 7, 1, 4, 3, 2] 75
[0, 6, 5, 8, 9, 7, 1, 4, 3, 2] 75
[0, 6, 5, 8, 9, 7, 1, 4, 3, 2] 75
[1, 7, 9, 8, 5, 6, 0, 4, 3, 2] 71
[1, 7, 9, 8, 5, 6, 0, 4, 3, 2] 71
[1, 7, 9, 8, 6, 5, 0, 4, 3, 2] 70
[1, 7, 9, 8, 6, 5, 0, 3, 4, 2] 68
[1, 7, 9, 8, 6, 5, 0, 3, 4, 2] 68
[1, 7, 9, 8, 6, 5, 0, 3, 4, 2] 68
divs
[1, 7, 9, 8, 6, 5, 2, 4, 3, 0] 68
[9, 7, 1, 8, 6, 5, 2, 4, 3, 0] 64
divs
[9, 7, 1, 8, 6, 4, 2, 5, 3, 0] 68
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6, 8, 1, 7, 0] 58
[9, 3, 5, 2, 4, 6,

([9, 2, 8, 5, 0, 1, 6, 7, 3, 4], 44, 137)

In [331]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [332]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [333]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification    
            new_value = calculate_total_edge_length(graph, new_solution)
            print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)
            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i
                value = new_value
                solution = deepcopy(new_solution)
    return solution, value, best_i

In [334]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 3, 5, 7, 6, 9, 2, 8, 4, 1] 71 0
[0, 3, 5, 7, 6, 9, 2, 8, 4, 1] 73 0
[0, 3, 5, 7, 6, 9, 2, 4, 8, 1] 72 1
[0, 3, 5, 7, 6, 9, 2, 4, 8, 1] 72 2
[0, 3, 5, 7, 6, 9, 2, 4, 8, 1] 72 3
[0, 3, 5, 7, 6, 9, 1, 8, 4, 2] 68 4
[0, 3, 5, 7, 6, 9, 1, 8, 4, 2] 68 5
[5, 3, 0, 7, 6, 9, 1, 8, 4, 2] 66 6
[5, 3, 0, 7, 6, 9, 1, 8, 4, 2] 66 7
[5, 3, 0, 7, 6, 9, 1, 8, 4, 2] 66 8
[5, 3, 0, 7, 6, 9, 1, 8, 4, 2] 66 9
vns:  [5, 3, 9, 6, 7, 0, 1, 8, 4, 2] 62 0
post shaking:  [0, 3, 9, 6, 7, 5, 1, 8, 2, 4] 74 0
[0, 3, 9, 6, 7, 5, 1, 8, 2, 4] 73 0
[1, 5, 7, 6, 9, 3, 0, 8, 2, 4] 72 1
[1, 5, 7, 0, 3, 9, 6, 8, 2, 4] 71 2
[5, 1, 7, 0, 3, 9, 6, 8, 2, 4] 70 3
[5, 1, 7, 0, 3, 6, 9, 8, 2, 4] 69 4
[5, 1, 7, 0, 3, 6, 9, 8, 2, 4] 69 5
[5, 1, 7, 0, 3, 6, 9, 8, 2, 4] 69 6
[5, 1, 2, 8, 9, 6, 3, 0, 7, 4] 63 7
[5, 1, 2, 8, 9, 6, 3, 0, 7, 4] 63 8
[6, 9, 8, 2, 1, 5, 3, 0, 7, 4] 58 9
vns:  [6, 9, 8, 2, 1, 5, 3, 0, 7, 4] 58 1
post shaking:  [6, 9, 8, 2, 1, 5, 4, 0, 7, 3] 62 1
[

([4, 3, 7, 6, 1, 8, 0, 5, 2, 9], 44, 12)

In [335]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 3, 5, 6, 7, 4, 8, 2, 9, 1] 68 0
[0, 3, 5, 6, 7, 4, 8, 2, 9, 1] 73 0
[6, 5, 3, 0, 7, 4, 8, 2, 9, 1] 71 1
[8, 4, 7, 0, 3, 5, 6, 2, 9, 1] 69 2
[8, 4, 7, 0, 3, 5, 6, 2, 9, 1] 69 3
[8, 4, 7, 0, 3, 5, 6, 2, 9, 1] 69 4
[0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 5
[0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 6
[0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 7
[0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 8
[0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 9
vns:  [0, 7, 4, 8, 3, 5, 6, 2, 9, 1] 65 0
post shaking:  [0, 7, 4, 2, 6, 5, 3, 8, 9, 1] 77 0
[0, 7, 4, 2, 6, 5, 3, 8, 9, 1] 73 0
[0, 8, 3, 5, 6, 2, 4, 7, 9, 1] 71 1
[5, 3, 8, 0, 6, 2, 4, 7, 9, 1] 68 2
[5, 3, 8, 0, 6, 2, 4, 7, 9, 1] 68 3
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 4
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 5
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 6
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 7
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 8
[5, 3, 8, 0, 6, 2, 4, 1, 9, 7] 64 9
vns:  [5, 3, 8, 0, 4, 2, 6, 1, 9, 7] 60 0
post shaking:  [5, 3, 8, 0, 4, 7, 9, 1, 6, 2] 62 0
[

([4, 3, 7, 6, 2, 1, 8, 0, 5, 9], 44, 62)

In [336]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 3, 5, 4, 7, 9, 2, 8, 6, 1] 62 0
[0, 3, 5, 4, 7, 9, 2, 8, 6, 1] 73 0
[0, 3, 5, 4, 7, 9, 8, 2, 6, 1] 61 1
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 2
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 3
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 4
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 5
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 6
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 7
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 8
[4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 9
vns:  [4, 3, 5, 0, 7, 9, 8, 2, 6, 1] 53 0
post shaking:  [4, 3, 5, 1, 7, 9, 2, 8, 6, 0] 66 0
[4, 3, 5, 1, 7, 9, 2, 8, 6, 0] 73 0
[4, 3, 5, 9, 7, 1, 2, 8, 6, 0] 58 1
[4, 3, 5, 9, 7, 1, 2, 8, 6, 0] 58 2
[4, 3, 5, 9, 7, 1, 0, 8, 6, 2] 54 3
[4, 3, 5, 9, 7, 1, 0, 8, 6, 2] 54 4
[4, 3, 5, 9, 7, 1, 0, 8, 6, 2] 54 5
[4, 3, 5, 6, 7, 1, 0, 8, 9, 2] 53 6
[4, 3, 5, 6, 7, 1, 0, 8, 9, 2] 53 7
[4, 3, 5, 6, 7, 1, 0, 8, 9, 2] 53 8
[4, 3, 7, 6, 5, 1, 0, 8, 9, 2] 49 9
vns:  [4, 3, 7, 6, 5, 1, 0, 8, 9, 2] 49 1
post shaking:  [4, 3, 7, 8, 5, 1, 0, 6, 9, 2] 57 1
[

([4, 3, 7, 6, 1, 2, 0, 8, 5, 9], 44, 34)

In [337]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [6, 5, 3, 0, 7, 9, 2, 8, 4, 1] 76 0
[6, 5, 3, 0, 7, 9, 2, 8, 4, 1] 73 0
[6, 5, 3, 0, 7, 9, 2, 8, 4, 1] 73 1
[6, 5, 3, 0, 7, 9, 2, 8, 4, 1] 73 2
[9, 5, 3, 0, 7, 6, 2, 8, 4, 1] 61 3
[9, 5, 3, 0, 7, 6, 2, 8, 4, 1] 61 4
[9, 5, 3, 8, 7, 6, 2, 0, 4, 1] 59 5
[9, 5, 3, 8, 7, 6, 2, 0, 4, 1] 59 6
[9, 5, 3, 8, 7, 6, 1, 0, 4, 2] 58 7
[9, 5, 3, 8, 7, 6, 1, 0, 4, 2] 58 8
[9, 5, 3, 8, 7, 6, 1, 0, 4, 2] 58 9
vns:  [9, 5, 3, 8, 7, 6, 1, 0, 4, 2] 58 0
post shaking:  [9, 5, 3, 8, 7, 6, 2, 4, 0, 1] 59 0
[9, 5, 3, 8, 7, 6, 2, 4, 0, 1] 73 0
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 1
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 2
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 3
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 4
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 5
[5, 9, 3, 8, 7, 6, 2, 4, 0, 1] 61 6
[9, 5, 3, 8, 7, 6, 2, 4, 0, 1] 59 7
[9, 5, 3, 8, 7, 6, 2, 4, 0, 1] 59 8
[9, 5, 3, 8, 7, 6, 2, 4, 0, 1] 59 9
vns:  [9, 5, 3, 8, 7, 6, 1, 0, 4, 2] 58 0
post shaking:  [9, 5, 3, 8, 7, 2, 4, 0, 1, 6] 64 0
[

([9, 5, 0, 8, 2, 1, 6, 7, 3, 4], 43, 57)

In [338]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 5, 3, 6, 7, 9, 2, 8, 4, 1] 71 0
[0, 5, 3, 6, 7, 9, 2, 8, 4, 1] 73 0
[0, 1, 3, 6, 7, 9, 2, 8, 5, 4] 71 1
[0, 1, 3, 6, 7, 9, 2, 8, 5, 4] 71 2
[0, 4, 3, 6, 7, 9, 2, 8, 5, 1] 65 3
[3, 4, 0, 6, 7, 9, 5, 1, 2, 8] 61 4
[3, 4, 0, 6, 7, 9, 5, 1, 2, 8] 61 5
[3, 4, 0, 6, 7, 9, 5, 1, 2, 8] 61 6
[3, 4, 0, 6, 7, 9, 5, 1, 2, 8] 61 7
[3, 4, 0, 6, 7, 2, 5, 8, 1, 9] 60 8
[3, 4, 0, 6, 7, 2, 5, 8, 1, 9] 60 9
vns:  [3, 4, 1, 6, 7, 2, 5, 8, 0, 9] 58 0
post shaking:  [3, 4, 1, 6, 8, 2, 9, 7, 0, 5] 66 0
[3, 4, 1, 6, 8, 2, 9, 7, 0, 5] 73 0
[4, 3, 1, 6, 8, 2, 9, 7, 0, 5] 64 1
[4, 3, 1, 6, 8, 2, 9, 7, 0, 5] 64 2
[4, 3, 1, 6, 8, 2, 9, 7, 0, 5] 64 3
[4, 3, 1, 6, 8, 7, 0, 2, 5, 9] 56 4
[4, 3, 1, 6, 8, 7, 0, 2, 5, 9] 56 5
[4, 3, 1, 6, 0, 7, 8, 2, 5, 9] 54 6
[4, 3, 1, 6, 0, 7, 8, 2, 5, 9] 54 7
[4, 3, 1, 6, 0, 7, 8, 2, 5, 9] 54 8
[4, 3, 1, 6, 0, 7, 8, 2, 5, 9] 54 9
vns:  [4, 3, 1, 6, 0, 7, 8, 2, 5, 9] 54 1
post shaking:  [4, 3, 0, 6, 1, 7, 8, 2, 5, 9] 50 1
[

([4, 3, 7, 6, 1, 0, 8, 5, 2, 9], 43, 59)

In [339]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 3, 5, 6, 7, 9, 1, 4, 8, 2] 71 0
[0, 3, 5, 6, 7, 9, 1, 4, 8, 2] 73 0
[7, 3, 5, 6, 0, 9, 1, 4, 8, 2] 63 1
[7, 3, 5, 6, 0, 1, 9, 4, 8, 2] 62 2
[7, 3, 5, 6, 0, 1, 9, 4, 8, 2] 62 3
[7, 3, 5, 6, 0, 1, 9, 4, 8, 2] 62 4
[7, 3, 5, 9, 0, 1, 6, 4, 8, 2] 61 5
[7, 3, 5, 9, 0, 1, 2, 6, 8, 4] 60 6
[7, 3, 5, 9, 0, 1, 2, 6, 8, 4] 60 7
[7, 3, 5, 9, 0, 1, 6, 2, 8, 4] 59 8
[7, 3, 5, 9, 0, 1, 6, 2, 8, 4] 59 9
vns:  [7, 3, 5, 9, 0, 1, 6, 2, 8, 4] 59 0
post shaking:  [7, 3, 5, 9, 0, 1, 4, 8, 2, 6] 61 0
[7, 3, 5, 9, 0, 1, 4, 8, 2, 6] 73 0
[6, 3, 5, 9, 0, 1, 7, 2, 8, 4] 71 1
[5, 3, 6, 9, 0, 1, 7, 2, 8, 4] 69 2
[5, 2, 6, 9, 0, 1, 7, 8, 3, 4] 68 3
[5, 2, 6, 9, 0, 1, 7, 8, 3, 4] 68 4
[1, 2, 6, 9, 0, 5, 7, 8, 3, 4] 63 5
[1, 2, 6, 8, 0, 5, 7, 9, 3, 4] 53 6
[1, 2, 6, 8, 0, 5, 7, 9, 3, 4] 53 7
[1, 2, 6, 8, 0, 5, 7, 9, 3, 4] 53 8
[1, 2, 6, 8, 0, 5, 7, 9, 3, 4] 53 9
vns:  [1, 2, 6, 8, 0, 5, 7, 9, 3, 4] 53 0
post shaking:  [1, 2, 6, 8, 0, 4, 3, 9, 7, 5] 59 0
[

([4, 3, 7, 6, 1, 2, 8, 0, 5, 9], 43, 48)

In [340]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 9, 5, 6, 7, 3, 2, 8, 4, 1] 71 0
[0, 9, 5, 6, 7, 3, 2, 8, 4, 1] 73 0
[1, 4, 8, 2, 3, 7, 6, 5, 9, 0] 71 1
[0, 9, 5, 6, 7, 3, 4, 1, 2, 8] 69 2
[0, 9, 5, 6, 7, 3, 4, 1, 2, 8] 69 3
[0, 9, 5, 7, 6, 3, 4, 1, 2, 8] 67 4
[4, 3, 6, 7, 5, 9, 0, 1, 2, 8] 55 5
[4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 6
[4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 7
[4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 8
[4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 9
vns:  [4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 0
post shaking:  [1, 3, 6, 2, 8, 0, 4, 7, 5, 9] 65 0
[1, 3, 6, 2, 8, 0, 4, 7, 5, 9] 73 0
[1, 3, 6, 2, 8, 7, 4, 0, 5, 9] 61 1
[1, 3, 6, 2, 8, 7, 4, 0, 5, 9] 61 2
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 3
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 4
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 5
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 6
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 7
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 8
[8, 2, 6, 3, 1, 7, 4, 0, 5, 9] 57 9
vns:  [4, 3, 6, 7, 8, 0, 1, 2, 5, 9] 49 1
post shaking:  [7, 3, 6, 4, 8, 0, 1, 2, 5, 9] 50 1
[

([4, 3, 7, 6, 0, 1, 8, 2, 5, 9], 44, 70)

In [341]:
vns(graph, random_solution, total_edge_length, 100, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

vns:  [0, 3, 5, 6, 7, 9, 2, 8, 4, 1] 73 0
post shaking:  [0, 3, 5, 6, 7, 9, 1, 4, 8, 2] 71 0
[0, 3, 5, 6, 7, 9, 1, 4, 8, 2] 73 0
[0, 3, 9, 7, 6, 5, 1, 8, 2, 4] 72 1
[0, 3, 7, 9, 6, 5, 1, 8, 2, 4] 71 2
[0, 3, 7, 9, 6, 5, 1, 8, 2, 4] 71 3
[0, 3, 7, 9, 6, 5, 1, 8, 2, 4] 71 4
[0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 5
[0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 6
[0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 7
[0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 8
[0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 9
vns:  [0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 0
post shaking:  [0, 4, 6, 5, 1, 8, 2, 9, 7, 3] 65 0
[0, 4, 6, 5, 1, 8, 2, 9, 7, 3] 73 0
[5, 6, 4, 0, 1, 8, 2, 9, 7, 3] 72 1
[8, 1, 0, 4, 6, 5, 2, 9, 7, 3] 63 2
[8, 1, 0, 4, 6, 5, 2, 9, 7, 3] 63 3
[8, 1, 0, 4, 6, 5, 2, 9, 7, 3] 63 4
[0, 1, 8, 4, 6, 5, 2, 9, 7, 3] 61 5
[0, 1, 8, 4, 6, 5, 2, 9, 7, 3] 61 6
[0, 1, 8, 4, 6, 2, 5, 9, 7, 3] 59 7
[0, 1, 8, 4, 6, 2, 5, 9, 7, 3] 59 8
[2, 6, 4, 8, 1, 0, 7, 3, 5, 9] 58 9
vns:  [0, 4, 2, 8, 1, 5, 6, 9, 7, 3] 57 0
post shaking:  [0, 4, 2, 8, 1, 3, 7, 9, 6, 5] 65 0
[

([9, 2, 5, 8, 0, 1, 6, 7, 3, 4], 43, 52)

In [342]:
#genetic algorithms

In [343]:
class Individual:
    def __init__(self, num_nodes, graph):
        self.num_nodes = num_nodes
        self.graph = graph
        self.code = self.generate_solution(self.num_nodes)
        self.fitness = self.calc_fitness(self.num_nodes, self.graph)
    
    def generate_solution(self, num_nodes):
        nodes = list(range(num_nodes))
        random.shuffle(nodes)
        return nodes

    def calc_fitness(self, num_nodes, graph):
        total_length = 0
        for i in range(len(graph)):
            for j in range(i + 1, len(graph)):
                if graph[i][j] == 1:
                    position_i = self.code.index(i)
                    position_j = self.code.index(j)
                    total_length += abs(position_i - position_j)
        return -total_length

In [344]:
i = Individual(10, graph)

In [345]:
i.code

[7, 4, 2, 3, 8, 6, 5, 9, 1, 0]

In [346]:
def tournament_selection(population, tour_size):
    tournament = random.sample(population, tour_size)
    selected = max(tournament, key=lambda x: x.fitness)
    return selected

In [347]:
# its like a wheel in a casino but instead of having equal proportions to
# for wheels slots we have different proportions(probabilities) for each slot
# its size depends on the fitness of an individual
# sum(i from 0 to n)pi where pi is probability of each one
def roulette_selection(population):
    total_fitness = sum(ind.fitness for ind in population)
    fitness_props = [ind.fitness / total_fitness for ind in population]
    cumulative_props = [sum(fitness_props[:i+1] for i in range(len(fitness_props)))]
    
    rand_prop = random.uniform(0,1)
    for i, prob in enumerate(cumulative_props):
        if rand_prop < prob:
            return population[i]

In [348]:
#beacause we are dealing with permutations, we have to be careful here and used sepcialized crossover algorithms
#ordered crossover method
#PROVERI OVO
def ordered_crossover(parent1, parent2, cx_point1 = -1, cx_point2 = -1):
    
    n = len(parent1.code)

    # Choose two random crossover points
    if cx_point1 == -1 and cx_point2 == -1:
        cx_point1, cx_point2 = sorted(random.sample(range(n), 2))
        print(cx_point1, cx_point2)

    # Copy the segment between the crossover points from parent1 to child1
    child1_segment = parent1.code[cx_point1:cx_point2 + 1]

    # Fill the remaining positions in child1 with elements from parent2
    child1 = [-1] * n
    child1[cx_point1:cx_point2 + 1] = child1_segment
    remaining_positions = [i for i in parent2.code if i not in child1_segment]
    j = 0
    for i in range(n):
        if child1[i] == -1:
            child1[i] = remaining_positions[j]
            j += 1

    return child1, cx_point1, cx_point2


In [350]:
def mutation_swap():
    pass

In [351]:
def mutation_inverse():
    pass

In [352]:
def pmx_crossover(parent1, parent2):
    n = len(parent1.code)
    start, end = sorted(random.sample(range(n), 2))
    
    child = [-1] * n
    child[start:end + 1] = parent1.code[start:end + 1]

    for i in range(start, end + 1):
        if parent2.code[i] not in child:
            #print(parent1.code)
            #print(parent2.code)
            index = parent2.code.index(parent1.code[i])
            #this next lines are the only difference between pmx and ordered
            #we dont change child1 in a ordered way but according to previous indexes
            while child[index] != -1:
                index = parent2.code.index(parent1.code[index])
            child[index] = parent2.code[i]
            
            
    for i in range(n):
        if child[i] == -1:
            child[i] = parent2.code[i]

    return child

In [367]:
#with linear ranking selection is not dependable on fitness but fitness rank
#this way worse individuals can get a chance to be choosen and maybe they can 
# give better results
def rank_selection_linear_ranking(population, num_selections,forbidden,selection_pressure = 1.5):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [(2-selection_pressure)/n + 2*i*(selection_pressure-1)/(n*(n-1)) for i in range(n)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)
    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [368]:

def rank_selection(population, num_selections, forbidden):
    n = len(population)
    ranked_population = sorted(population, key = lambda x: x.fitness, reverse=True)
    
    probs = [i/n for i in range(1, n + 1)]
    
    selection = random.choices(ranked_population, weights=probs, k = num_selections)

    max_fitness = float('-inf')
    best_i = -1
    for i in range(len(selection)):
        if population[i].fitness > max_fitness and i != forbidden:
            max_fitness = population[i].fitness
            best_i = i
    
    return best_i

In [375]:
def mutation(individual, mutation_prob):
    for i in range(len(individual.code)):
        if random.random() < mutation_prob:
            rand_i = random.choice(range(len(individual.code)))
            tmp = individual.code[i]
            individual.code[i] = individual.code[rand_i]
            individual.code[rand_i] = tmp

In [376]:
def genetic_algo(population_size, graph, num_nodes,num_generations,tournament_size, mutation_prob, elitism_size, crossover=pmx_crossover, selection=rank_selection):
    #if use_elitism and (population_size - elitism_size) % 2 == 1:
    #    elitism_size += 1
    
    population = [Individual(num_nodes, graph) for _ in range(population_size)]
    new_population = population.copy()
    
    for i in range(num_generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        for j in range(elitism_size, population_size, 2):
            parent1_i = selection(population, tournament_size, forbidden=-2)
            parent2_i = selection(population, tournament_size, parent1_i)
            new_population[j].code = crossover(population[parent1_i], population[parent2_i])
            new_population[j+1].code = crossover(population[parent1_i], population[parent2_i])
        
            mutation(new_population[j], mutation_prob)
            mutation(new_population[j+1], mutation_prob)
            
            new_population[j].fitness = new_population[j].calc_fitness(num_nodes, graph)
            new_population[j+1].fitness = new_population[j+1].calc_fitness(num_nodes, graph)
            
        population = new_population.copy()
    return max(population, key=lambda x: x.fitness)

In [377]:
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=3, elitism_size=4,mutation_prob=0.05, crossover=pmx_crossover)

In [378]:
best_individual.fitness

-43

In [379]:
#sa linearnim rank selectionom
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=3, elitism_size=4,mutation_prob=0.05, crossover=pmx_crossover, selection=rank_selection_linear_ranking)

In [380]:
best_individual.fitness

-47

In [381]:
#iz nekog razloga ne radi sa ordered_crossoverom?????
best_individual = genetic_algo(population_size=100, graph=graph, num_nodes=num_nodes, num_generations=10, tournament_size=3, elitism_size=4,mutation_prob=0.05, crossover=ordered_crossover)

2 9
2 3


ValueError: tuple.index(x): x not in tuple